In [ ]:
# パッケージのインストール
%pip install -q langchain
%pip install -q langchain-openai
%pip install -q langchain-chroma
%pip install -q langchain-community
%pip install -q python-dotenv
%pip install -q pdfminer.six
%pip install -q chromadb
%pip install -q ipywidgets
%pip install -q Jinja2
%pip install -q pandas
%pip install -q ragas datasets
%pip install -q matplotlib


In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [6]:
chunk_size = 200
collection_name = f"mycollection_{chunk_size}_ef_100"
embedding_model = "text-embedding-3-large"

top_k = 10
batch_size = 500

query="高町なのはがユーノから譲り受けたものは何ですか？"

### 埋め込みモデルの準備

In [7]:
from langchain_openai import OpenAIEmbeddings

# 埋め込みモデルの準備
embeddings = OpenAIEmbeddings(model=embedding_model)

### ドキュメントの読み込みとチャンキング

In [8]:
from langchain_community.document_loaders import PDFMinerLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# ドキュメントの読み込み
loader = DirectoryLoader(
    "./data/",
    glob="**/*.pdf",  # PDFファイルを対象に
    show_progress=True,
    loader_cls=PDFMinerLoader  # PDFMinerLoaderを使用
)

In [ ]:
import unicodedata

# ドキュメントの分割
documents = []
for doc in loader.load():
    # すべての改行を削除（単語の途中でチャンキングされることを防ぐため）
    doc.page_content = unicodedata.normalize('NFKC', doc.page_content.replace("\n", ""))
    chunks = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_size // 10,
        separators=[
            "。",
            "、",
            " ",
            "」",
            "）",
            "｝",
            "〕",
            "》"
        ],
        keep_separator="end"
    ).split_documents([doc])
    documents.extend(chunks)

print(f"合計チャンク数: {len(documents)}")

In [ ]:
for i, doc in enumerate(documents[:5]):
    print(f"\n=== Document {i+1} ===")
    print(f"Content: {doc.page_content}")
    print(f"文字数: {len(doc.page_content)}")
    print(f"Source: {doc.metadata['source']}")

### VectorStoreの準備

In [ ]:
import chromadb
from langchain_chroma import Chroma


# Chromaクライアントを初期化
try:
    persistent_client = chromadb.PersistentClient(path="./chroma_db")
    # 既存のChromaコレクションを削除
    try:
        persistent_client.delete_collection(collection_name)
        print(f"既存のコレクション '{collection_name}' を削除しました")
    except ValueError as e:
        print(f"既存のコレクション '{collection_name}' は存在しませんでした")
    except Exception as e:
        print(f"コレクションの削除中にエラーが発生しました: {str(e)}")

    vector_store = Chroma(
        client=persistent_client,
        collection_name=collection_name,
        embedding_function=embeddings,
        collection_metadata={"hnsw:space": "ip", "hnsw:search_ef": 100} 
    )
    print(f"コレクション '{collection_name}' を作成しました")

    collections = persistent_client.list_collections()
    print("現在のコレクション一覧:")
    for collection in collections:
        print(f"- {collection.name}")

    # ドキュメントをバッチで追加
    print(f"コレクション '{collection_name}' にデータをロードします")
    total_added = 0
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i + batch_size]
        vector_store.add_documents(documents=batch, embedding=embeddings)
        total_added += len(batch)
        print(f"バッチ {i//batch_size + 1} を追加しました（{len(batch)}件）")
    
    print(f"\n合計 {total_added} 件のドキュメントを追加しました")

except Exception as e:
    print(f"Chromaデータベースの操作中にエラーが発生しました: {str(e)}")


### ベクトル検索のテスト

In [12]:
result = vector_store.similarity_search(query, k=top_k)

In [ ]:
for i, doc in enumerate(result[:top_k]):
    print(f"\n=== 検索結果 {i+1} ===")
    print(f"Content: {doc.page_content}")
    print(f"文字数: {len(doc.page_content)}")
    print(f"Source: {doc.metadata['source']}")